In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Kích thước ảnh
img_rows = 224
img_cols = 224
nb_class = 1
hidden_dim = 512
batch_size = 4
epochs = 10

# Tải mô hình VGG16 không bao gồm lớp FC (Fully Connected) phía trên
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))

# Đóng băng 4 lớp cuối cùng của mô hình cơ sở
for layer in base_model.layers:
    layer.trainable = False

# In các lớp để kiểm tra
# for (i, layer) in enumerate(base_model.layers):
#     print(f"{i} {layer.__class__.__name__} {layer.trainable}")

# Thêm các lớp phía trên mô hình cơ sở
top_model = base_model.output
top_model = Flatten(name="flatten")(top_model)
top_model = Dense(32, activation="relu")(top_model)
top_model = Dropout(0.3)(top_model)
top_model = Dense(1, activation="sigmoid")(top_model)  # Sử dụng sigmoid cho phân loại nhị phân

# Tạo mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=top_model)

# Biên dịch mô hình
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator = train_datagen.flow_from_directory(
    'C:/Users/ducth/Downloads/Face_Recognition/Facenet/Data/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'C:/Users/ducth/Downloads/Face_Recognition/Facenet/Data/val',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')



Found 90 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
r=validation_generator.class_indices
s=train_generator.class_indices
print(r)
print(s)

{'Empty': 0, 'Thang': 1}
{'Empty': 0, 'Thang': 1}


In [ ]:
# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

<ipython-input-11-bfb9036b067c>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
22/22 [==============================] - 56s 2s/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 1.2473e-04 - val_accuracy: 1.0000
Epoch 2/10
22/22 [==============================] - 55s 2s/step - loss: 4.1243e-04 - accuracy: 1.0000 - val_loss: 5.4693e-05 - val_accuracy: 1.0000
Epoch 3/10
22/22 [==============================] - 57s 3s/step - loss: 2.4802e-04 - accuracy: 1.0000 - val_loss: 3.8691e-05 - val_accuracy: 1.0000
Epoch 4/10
22/22 [==============================] - 54s 2s/step - loss: 3.1259e-04 - accuracy: 1.0000 - val_loss: 3.6894e-05 - val_accuracy: 1.0000
Epoch 5/10
22/22 [==============================] - 54s 2s/step - loss: 6.1169e-04 - accuracy: 1.0000 - val_loss: 2.4681e-05 - val_accuracy: 1.0000
Epoch 6/10
22/22 [==============================] - 52s 2s/step - loss: 3.2234e-04 - accuracy: 1.0000 - val_loss: 2.0640e-05 - val_accuracy: 1.0000
Epoch 7/10
22/22 [==============================] - 56s 3s/step - loss: 4.4996e-04 - accuracy: 1.0000 - val_loss: 1.

In [ ]:
# Save the model
import os
save_dir = '/content/drive/MyDrive/Colab Notebooks/Facenet/Pretrain'
save_path = os.path.join(save_dir, 'custom_vggface_model_thang.h5')
model.save(save_path)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Hàm để chuẩn bị ảnh cho dự đoán
def prepare_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)  # Tải ảnh và thay đổi kích thước
    image = img_to_array(image)  # Chuyển ảnh thành mảng numpy
    image = np.expand_dims(image, axis=0)  # Thêm chiều batch
    image = image / 255.0  # Chuẩn hóa pixel
    return image

# Ví dụ: Chuẩn bị một ảnh mới
new_image_path = '/content/drive/MyDrive/Colab Notebooks/Facenet/Data/val/Thang/14d35c69-image_92_crop_0.jpg'
image = prepare_image(new_image_path)


In [ ]:
# Dự đoán
prediction = model.predict(image)
print(prediction)

# In ra kết quả dự đoán
if prediction[0][0] > 0.5:
    print("Ảnh này là của Thang.")
else:
    print("Ảnh này không phải của Thang.")

1/1 [==============================] - 0s 477ms/step
[[0.9999902]]
Ảnh này là của Thang.


In [17]:
from keras.models import load_model
import os

# Đường dẫn tới mô hình đã lưu
save_dir = '/content/drive/MyDrive/Colab Notebooks/Facenet/Pretrain'
save_path = os.path.join(save_dir, 'custom_vggface_model_thang.h5')

# Tải mô hình
model = load_model(save_path)
new_image_path = '/content/drive/MyDrive/Colab Notebooks/Facenet/Data/val/Thang/14d35c69-image_92_crop_0.jpg'
image = prepare_image(new_image_path)
# Dự đoán
prediction = model.predict(image)
print(prediction)

# In ra kết quả dự đoán
if prediction[0][0] > 0.5:
    print("Ảnh này là của Thang.")
else:
    print("Ảnh này không phải của Thang.")

1/1 [==============================] - 1s 626ms/step
[[0.9999902]]
Ảnh này là của Thang.
